In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Import OSNIR Dataset and shuffle data

In [2]:
# import OSNIR dataset
df = pd.read_csv('../Data/OSNIR_values_extendedv3_new datasetbcsv.csv')
# shuffling rows of OSNIR dataframe and reset indexes
df = df.sample(frac=1)
df.reset_index(inplace=True, drop=True)

Preview of first 5 samples

In [3]:
df.head()

,OSNIRnumerical(dB),Ns,Pch(dBm),L(km),B(GHz),GB(GHz),Nch
0,19.822029,28,0,10,25.0,0.0,3
1,10.728774,28,3,100,50.0,0.0,9
2,26.215406,20,-2,10,50.0,0.0,9
3,21.377177,7,-6,50,50.0,0.0,9
4,28.853978,10,-3,10,50.0,0.0,9


1800 values for training (0,1800-1)\
600 values for validation (1800,2400-1)\
600 values for testing (2400,3000-1)

In [4]:
data = np.array(df)

In [5]:
# input x aka Ns, Pch, L, B, GB, Nch values
x_train = data[0:1800-1, 1:7]
x_valid = data[1800:2400-1, 1:7]
x_test = data[2400:3000-1, 1:7]

# output y aka OSNIR values
y_train = data[0:1800-1, 0]
y_valid = data[1800:2400-1, 0]
y_test = data[2400:3000-1, 0]

In [6]:
x_train

array([[ 28. ,   0. ,  10. ,  25. ,   0. ,   3. ],
       [ 28. ,   3. , 100. ,  50. ,   0. ,   9. ],
       [ 20. ,  -2. ,  10. ,  50. ,   0. ,   9. ],
       ...,
       [ 19. ,   3. ,  10. ,  25. ,  25. ,   9. ],
       [ 41. ,  -3. ,  10. ,  12.5,   0. ,   9. ],
       [ 11. ,  -5. ,  10. ,  50. ,   0. ,   9. ]])

### Create the Neural Network
1 Input layer, 2 hidden layers (32 neurons each) and 1 Output Layer

In [7]:
model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(6,), activation='relu'),  # first hidden layer
    keras.layers.Dense(32, activation='relu'),  # second hidden layer
    keras.layers.Dense(1, activation='relu') # output layer (3)
])

Early hault of training if loss has not improved in 50 epochs in validation set

In [11]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=50)

Adam Optimizer, Mean Square Error Loss

In [12]:
model.compile(optimizer='adam',
              loss='mse'
              #metrics=['accuracy']
              )

Fitting the model, run for 500 epochs

In [13]:
model.fit(x_train, y_train,
          validation_data=(x_valid,y_valid), 
          epochs=500, 
          batch_size=25,
          callbacks=[early_stop])

Epoch 1/500
72/72 [==============================] - 1s 3ms/step - loss: 0.4777 - val_loss: 0.1682
Epoch 2/500
72/72 [==============================] - 0s 3ms/step - loss: 0.1231 - val_loss: 0.1727
Epoch 3/500
72/72 [==============================] - 0s 2ms/step - loss: 0.1472 - val_loss: 0.1690
Epoch 4/500
72/72 [==============================] - 0s 2ms/step - loss: 0.1742 - val_loss: 0.1586
Epoch 5/500
72/72 [==============================] - 0s 2ms/step - loss: 0.1584 - val_loss: 0.3798
Epoch 6/500
72/72 [==============================] - 0s 2ms/step - loss: 0.2686 - val_loss: 0.3247
Epoch 7/500
72/72 [==============================] - 0s 2ms/step - loss: 0.1948 - val_loss: 0.1808
Epoch 8/500
72/72 [==============================] - 0s 2ms/step - loss: 0.2067 - val_loss: 0.4823
Epoch 9/500
72/72 [==============================] - 0s 2ms/step - loss: 0.2195 - val_loss: 0.2234
Epoch 10/500
72/72 [==============================] - 0s 2ms/step - loss: 0.1737 - val_loss: 0.2219
Epoch 11/

In [14]:
predictions = (model.predict(x_test)) 

19/19 [==============================] - 0s 1ms/step


In [15]:
for i in range(10):
    print('%s => %f (expected %f)' % (x_test[i].tolist(), predictions[i], y_test[i] ))

[12.0, -6.0, 10.0, 25.0, 0.0, 9.0] => 28.322145 (expected 28.341298)
[23.0, 0.0, 100.0, 25.0, 0.0, 9.0] => 12.754276 (expected 12.238082)
[5.0, -1.0, 10.0, 12.5, 0.0, 9.0] => 23.247171 (expected 23.861305)
[35.0, 0.0, 100.0, 25.0, 0.0, 15.0] => 9.890615 (expected 9.848776)
[47.0, -4.0, 100.0, 12.5, 0.0, 9.0] => 10.203746 (expected 9.799840)
[25.0, 0.0, 10.0, 25.0, 0.0, 3.0] => 20.377577 (expected 20.360340)
[6.0, 3.0, 10.0, 25.0, 12.5, 9.0] => 23.592607 (expected 23.958640)
[14.0, -1.0, 50.0, 25.0, 0.0, 9.0] => 19.875135 (expected 19.840868)
[24.0, -7.0, 50.0, 25.0, 0.0, 3.0] => 17.906097 (expected 18.010393)
[10.0, -5.0, 50.0, 25.0, 0.0, 9.0] => 23.469854 (expected 23.022427)
